# LLM Observability Demo
This notebook runs a simple `LangGraph` agent that uses custom tools capable of querying the ESPN API for NFL data.

The `LangGraph` traces are automatically logged to both the `LangSmith` and `Arize` observability platforms without the need for annotations or any explicit code. Simply set the platform-specific environment variables. From my experience, traces are captured immediately in `LangSmith` and take a few minutes to reflect in `Arize`.

This demo uses `OpenAI` but could easily use `Anthropic` by replacing `langchain_openai` with `langchain_anthropic`.

References:
- [LangSmith / Tracing / LangGraph](https://docs.smith.langchain.com/observability/how_to_guides/tracing/trace_with_langgraph)
- [Arize / Tracing Integrations / LangChain](https://docs.arize.com/arize/llm-tracing/tracing-integrations-auto/langchain)

## Setup

### Install the dependencies

In [1]:
# Install the Langchain stack
%pip install -qq -U langgraph langsmith langchain_openai

# Install the Arize dependencies
%pip install -qq -U arize-otel openinference-instrumentation-langchain

# Install dotenv to manage env variables
%pip install -qq python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Set the environment variables
The following environment variables are required and loaded from a `.env` file:

- OPENAI_API_KEY
- LANGCHAIN_TRACING_V2
- LANGCHAIN_ENDPOINT
- LANGCHAIN_API_KEY
- LANGCHAIN_PROJECT
- ARIZE_API_KEY
- ARIZE_SPACE_ID
- ARIZE_PROJECT_NAME

In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

True

### Configure Arize with OpenTelemetry
While `LangSmith` integrates with `LangGraph` natively (i.e., simply requires the environment variables to be set), `Arize` requires this addditional setup step.

In [3]:
# Import open-telemetry dependencies
from arize.otel import register

# Setup OTEL via our convenience function
tracer_provider = register(
    space_id = os.getenv('ARIZE_SPACE_ID'), # in app space settings page
    api_key = os.getenv('ARIZE_API_KEY'), # in app space settings page
    project_name = os.getenv('ARIZE_PROJECT_NAME'), # name this to whatever you would like
)

# Import the automatic instrumentor from OpenInference
from openinference.instrumentation.langchain import LangChainInstrumentor

# Finish automatic instrumentation
LangChainInstrumentor().instrument(tracer_provider=tracer_provider)

/Users/williamcougan/Desktop/workspace/stats-copilot/venv2/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


🔭 OpenTelemetry Tracing Details 🔭
|  Arize Project: STATS_AGENT
|  Span Processor: BatchSpanProcessor
|  Collector Endpoint: otlp.arize.com
|  Transport: gRPC
|  Transport Headers: {'space_id': '****', 'api_key': '****', 'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.



Transient error StatusCode.DEADLINE_EXCEEDED encountered while exporting traces to otlp.arize.com, retrying in 1s.
Transient error StatusCode.DEADLINE_EXCEEDED encountered while exporting traces to otlp.arize.com, retrying in 1s.
Transient error StatusCode.DEADLINE_EXCEEDED encountered while exporting traces to otlp.arize.com, retrying in 2s.
Transient error StatusCode.DEADLINE_EXCEEDED encountered while exporting traces to otlp.arize.com, retrying in 4s.


## Main: ESPN NFL Stats Agent

### Build the agent

In [4]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

from espn_tools import get_player_id, get_player_stats, get_team_id, get_team_players, get_nfl_team_stats, get_recent_game_stats

tools = [get_player_id, get_player_stats, get_team_id, get_team_players, get_nfl_team_stats, get_recent_game_stats]
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools(tools)

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=tools)
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

### Run the agent

In [5]:
user_input = "How many touchdowns did Josh Allen of the Buffalo Bills average per game this season?"

config = {"configurable": {"thread_id": "JOSH-ALLEN-TDS"}}

events = graph.stream({"messages": [("user", user_input)]}, config, stream_mode="values")

for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

How many touchdowns did Josh Allen of the Buffalo Bills average per game this season?
================================== Ai Message ==================================
Tool Calls:
  get_team_id (call_cjbq7lZOosFdMY8IImap2Es0)
 Call ID: call_cjbq7lZOosFdMY8IImap2Es0
  Args:
    team_abbreviation: BUF
================================= Tool Message =================================
Name: get_team_id

2
================================== Ai Message ==================================
Tool Calls:
  get_team_players (call_FStL9ik2FvcTK14XtTzloXTU)
 Call ID: call_FStL9ik2FvcTK14XtTzloXTU
  Args:
    team_id: 2
================================= Tool Message =================================
Name: get_team_players

[{"id": "3918298", "uid": "s:20~l:28~a:3918298", "guid": "853f8768-54bd-6a4f-cb10-63d6df1e7742", "alternateIds": {"sdr": "3918298"}, "firstName": "Josh", "lastName": "Allen", "fullName": "Josh Allen", "di